In [ ]:
import pandas as pd 
import numpy as np 
import librosa
import soundfile as sf

In [ ]:
data = [ 
    {"audio": '../../data/audio_27092022_1.WAV', "lidar_log": "../../data/lidar_data_with_audio_timestamps_27_09_22_1.csv"},
    {"audio": '../../data/audio_27092022_2.WAV', "lidar_log": "../../data/lidar_data_with_audio_timestamps_27_09_22_2.csv"},
    ]
SR=16000

In [ ]:
def get_audio(path):
    print("Started loading audio")
    audio_clips = []

    audio, sr = librosa.load(path, sr = SR)
    audio_clips.append((audio))
    print(len(audio), "done")
    return audio

In [ ]:
def split_audio(audio, timestamps, window, audio_file_path):
    
    data1 = timestamps.copy()
    data1["filename"] = None
    for i, row in timestamps.iterrows():
        height, width,duration,  class_1, class_2, start, end = row.values
        start = int((start-window)*SR)
        end = int((end+window)*SR)
        sf.write('../../data/splitted_audio/{}_{}.wav'.format(audio_file_path, i), audio[start:end], SR, 'PCM_24')
        data1.loc[i, 'filename'] = '{}_{}.wav'.format(audio_file_path, i)
    return data1

In [ ]:
SR=32000
def create_dataset():
    start_index=0
    k = 0
    for dataset in data:
        audio_file_path = dataset["audio"]
        y, sr = librosa.load(audio_file_path, sr=SR)
        audio_length = librosa.get_duration(y=y, sr=sr)
        
        lidar_log = pd.read_csv(dataset["lidar_log"])
        lidar_log = lidar_log[["height", "width", "duration", "class_1", "class_2", "audio_start_s", "audio_end_s"]]
        lidar_log = lidar_log[(lidar_log["audio_start_s"] >= 0) & (lidar_log["audio_end_s"] <= audio_length)]
        
        print(lidar_log.shape)
        files = split_audio(y,lidar_log, window=0.25, audio_file_path=audio_file_path)
        start_index += len(lidar_log)
        if not isinstance(k, pd.DataFrame):
            k = pd.DataFrame(lidar_log)
        else:
            k = pd.concat([k, lidar_log], axis=0)
        
        print(k.shape)

    k.to_csv("../../data/splitted_audio/full_data_270922.csv")

create_dataset()

(440, 7)
(440, 7)
(824, 7)
(1264, 7)
